In [25]:
# Might need to dockerize later

# %pip install pandas
# %pip install numpy
# %pip install translate
# %pip install nltk
# %pip install openpyxl

In [26]:
import pandas as pd
import numpy as np
import nltk
# nltk.download('punkt')
import csv
csv_filename = "root_words.csv"
from translate import Translator
translator = Translator(from_lang="bn",to_lang="en")

In [27]:
def LCstring(X, Y):
    m=len(X)
    n=len(Y)
    LCSuff = [[0 for k in range(n+1)] for l in range(m+1)]
    result = 0
    for i in range(m + 1):
        for j in range(n + 1):
            if (i == 0 or j == 0):
                LCSuff[i][j] = 0
            elif (X[i-1] == Y[j-1]):
                LCSuff[i][j] = LCSuff[i-1][j-1] + 1
                result = max(result, LCSuff[i][j])
            else:
                LCSuff[i][j] = 0
    return result

In [28]:
def LCsequence(X, Y):
    # find the length of the strings
    m = len(X)
    n = len(Y)

    # declaring the array for storing the dp values
    L = [[None]*(n + 1) for i in range(m + 1)]

    """Following steps build L[m + 1][n + 1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif X[i-1] == Y[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])
    return L[m][n]


In [29]:
#fetch suffix
suffix=pd.read_excel('suffix_set.xlsx')
class1_suffix=suffix[suffix['weight']==1]
class2_suffix=suffix[suffix['weight']==2]
class3_suffix=suffix[suffix['weight'].isin([1,3])]
# hybrid_suffix=suffix[suffix['weight'].isin([3,4])]

In [30]:
dictnary=pd.read_excel('root_words.xlsx')
def slice_dictnary(test_word):
    # if len(test_word)>2 and not (class1_suffix['suffix'].isin([test_word[1]]).any()):
    #     selector=2
    # else:
    #     selector=1
    selector=1
    if(test_word[0]=='উ'):
        prefix1=test_word[:selector]
        prefix2='ও'+test_word[1:selector]
        sub_dictnary=dictnary[(dictnary['word'].str.match(f'{prefix1}+'))|(dictnary['word'].str.match(f'{prefix2}+'))]
    elif(test_word[0]=='এ'):
        prefix1=test_word[:selector]
        prefix2='আ'+test_word[1:selector]
        sub_dictnary=dictnary[(dictnary['word'].str.match(f'{prefix1}+'))|(dictnary['word'].str.match(f'{prefix2}+'))]
    elif(test_word[0]=='ঐ'):
        prefix1=test_word[:selector]
        prefix2='ই'+test_word[1:selector]
        sub_dictnary=dictnary[(dictnary['word'].str.match(f'{prefix1}+'))|(dictnary['word'].str.match(f'{prefix2}+'))]
    else:
        prefix=test_word[:selector]
        sub_dictnary=dictnary[(dictnary['word'].str.match(f'{prefix}+'))]
    return(sub_dictnary)


In [31]:
def checking(test_word):
    global prediction_list
    global sub_dictnary
    if sub_dictnary['word'].isin([test_word]).any():
        prediction_list.append(test_word)
    else:
        return(True)

In [32]:
def checking_with_translation(left_context,test_word,right_context):
    root=[0.2,0.0,'']
    translation_word=translator.translate(test_word).lower()
    actual_window=left_context+" "+test_word+" "+right_context
    translation_window=translator.translate(actual_window).lower()
    while not root[2]:
        for predicted_word in prediction_list:
            error=(len(test_word)+len(predicted_word)-2*LCsequence(test_word,predicted_word))/(len(test_word)+len(predicted_word))
            if error<=root[0]: 
                test_window=left_context+" "+predicted_word+" "+right_context
                translation_testwindow=translator.translate(test_window).lower()
                accurecy=LCsequence(translation_window,translation_testwindow)/len(translation_testwindow)
#               print(translation_testwindow,accurecy)
                if accurecy>root[1]:
                    root[0]=error
                    root[1]=accurecy
                    root[2]=predicted_word
                elif accurecy==root[1]:
                    if(LCsequence(test_word,root[2])<=LCsequence(test_word,predicted_word)):
                        root[0]=error
                        root[1]=accurecy
                        root[2]=predicted_word
        if not root[2]:
            root[0]+=0.1
                
    return(root[2])


In [33]:
def predict_root(target_suffix_set,test_word):
    predicted_root=[test_word+suffix for suffix in target_suffix_set.suffix]
    for i in predicted_root:
        if checking(i):
            continue
    return predicted_root

In [34]:
def remove_suffix(target_suffix,word):
    final_suffix=''
    #find actual suffix 
    #find longest suffix bettwen target_suffix that belongs at the end of given word 
    SL_word=[]
    for i in target_suffix:
        count=0
        for j in range(-1,(-(len(i)+1)),-1):
            if (word[j]==i[j]):
                
                count+=1
            else:
                break
        if(count==len(i)):
            final_suffix=i
            SL_word.append(word.rstrip(final_suffix))
    return(SL_word)

In [35]:
def add_to_dataset(context_word, root_word):
    with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        rows = [row for row in reader]

        # Find the row with the root word and add the related word to it
        found_root_word = False
        for row in rows:
            if row[0] == root_word:
                row.append(context_word)
                found_root_word = True
                break

        # Write the modified list back to the CSV file
        if found_root_word:
            with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(rows)
                print(f"Added {context_word} as a related word for {root_word}")
        else:
            print(f"{root_word} not found in the dataset")

In [36]:
def leematizer(context_word):
    global prediction_list
    container=[]
    #print(sub_dictnary)
    container.append(context_word)
    print("input ",context_word)
    if(context_word[0]=='উ'):
        container.append('ও'+context_word[1:])
        #print(container)
    elif(context_word[0]=='এ'):
        container.append('আ'+context_word[1:])
    elif(context_word[0]=='ঐ'):
        container.append('ই'+context_word[1:])
    for word in container:
        checking(word)
        #checking for direct root
        # if (class1_suffix['suffix'].isin([word[-1]]).any()):
        #     #if there is any suffix of class 1 remove it
        #     SL_word=remove_suffix(word[-1],word)
        #     # SL_word denoted suffix_less_word
        #     checking(SL_word)
        #     predict_root(class1_suffix,SL_word)
        index=[i in word for i  in class3_suffix.suffix]
        # select class 3 suffix
        if any(index):
            # sort target suffix list in desc order of length
            target_suffix=class3_suffix[index].suffix.tolist()
            target_suffix.sort(key=len,reverse=True)
            #print(target_suffix)
            SL_word_set=remove_suffix(target_suffix,word)
            if SL_word_set:
                #remove suffix
                #SL_word denoted suffix_less_word
                for SL_word in SL_word_set:
                    checking(SL_word)
                    predict_root(class2_suffix,word) # select only class 1 suffix suffix that is in the word
                    index=[]
                    #print(SL_word)
                    if len(SL_word)>1 and (class1_suffix['suffix'].isin([SL_word[-1]]).any()): #remove class 1 suffix suffix from end
                        S_word=SL_word.rstrip(SL_word[-1])
                    else:
                        S_word=SL_word
                    checking(S_word)
                    #if Sl_word is root word declare it or checck it with other suffix one by one
                    predict_root(class2_suffix,S_word)
                    predicted_subword=predict_root(class1_suffix,S_word)
                    #print(predicted_subword)
                    #if it is a root word declear rootword else check is there any other suffix present
                    #container.append(word)
                    for i in predicted_subword:
                        predict_root(class2_suffix,i)
    prediction_list=[*set(prediction_list)]
    #remove duplicate prediction 
    #print(prediction_list)
    if(len(prediction_list)==1): #if predicttion list contain only one prediction declear as root
        print(context_word,'--->',prediction_list[0])
        add_to_dataset(context_word, prediction_list[0])
    elif(len(prediction_list)>1): #if more than one check using meaning
        root=checking_with_translation(left_context,context_word,right_context)
        print(context_word,'--->',root)
        add_to_dataset(context_word, prediction_list[0])
    else:
        print(context_word,"not in database")
    #     root=[0,'']
    #     for dict_word in sub_dictnary.word:
    #         lcs=LCsequence(dict_word,context_word)
    #         if(lcs>root[0]):
    #             root[0]=lcs
    #             root[1]=dict_word
    #     print(context_word,'--->',root[1])
            


In [37]:
words=[]
#sentence1="ছাত্রজীবন মানব জীবনের প্রস্তুতিকাল। ছাত্রজীবনের ভিত্তির উপর ধীরে ধীরে মানবজীবনের কর্ম কৃতিত্বের সৌধটি গড়ে ওঠে। এই ভিত্তি সুদৃঢ় হবে, ভবিষ্যতে জীবন ও তত বলিষ্ঠ এবং সমৃদ্ধ হবে। মানব জীবনের ভিত্তি সুদৃঢ় করতে হলে ছাত্রজীবনে যত্নবান হতে হবে।তাই ছাত্রজীবনে লেখাপড়াও খেলাধুলার সঙ্গে সঙ্গে যেমন শরীর ও চরিত্র গঠন করতে হবে, তেমনি জগৎ ও জীবনের বহু বিচিত্র কর্মধারার সঙ্গে পরিচয় সাধন করতে হবে। এই পরিচয় সাধনের সূত্রই ছাত্রজীবনে সমাজকল্যাণ অথবা সমাজসেবা।"
#sentence2="সভ্যতার প্রথম উন্মেষ কালে আগুন আবিষ্কারের পরই সমাজ গঠন বোধকরি মানব সভ্যতার অন্যতম অবদান। তারপর থেকে সভ্যতার অগ্রগতি ও ক্রমবিকাশের সঙ্গে সঙ্গে গোষ্ঠীবদ্ধ মানুষ কতগুলি অলিখিত নিয়ম কানুন ও রীতিনীতি সূত্রে একে অন্যের সঙ্গে পরস্পর গাঁতা হয়ে পৃথিবীর এক এক অঞ্চলে একত্রে বসবাস করতে আরম্ভ করলো। এই ভাবেই একদিন সুসভ্য মানুষের সমাজ গড়ে উঠলো। সমাজের উদ্ভবের প্রথম দিন থেকেই মানুষ সমাজ সেবায় প্রয়োজন অনুভব করে। আজ সভ্যতার চরম উৎকর্ষের দিনেও মানব সমাজে এই সেবা ও সহযোগিতার প্রয়োজন এতটুকু কমেনি। সুতরাং সমাজসেবা বলতে সমাজের অন্তর্গত মানুষের সেবা ও কল্যাণকেই বোঝায়।"
#sentence3="সমাজসেবার প্রয়োজন সব দেশে থাকলেও ভারতবর্ষের মতো দরিদ্র উন্নয়নশীল বিশাল দেশে এর গুরুত্ব সমাধি। ১০০ কোটি জন অধ্যুষিত এই বিশাল দেশের কোটি কোটি মানুষ নিদারুণ দুঃখ দুর্দশা ও অভাব-অনটনে নিপীড়িত। দেশের অনেক মানুষই আজও নিরক্ষরতার নিবিড় অন্ধকারে নিমজ্জিত। এর ওপর আছে প্রাকৃতিক বিপর্যয় ঝড়-ঝঞ্ঝা খরা, বন্যা, ভূমিকম্প, ও দুর্ভিক্ষ মহামারীর তান্ডব লীলা। বলা বাহুল্য, এদেশে এসব সমস্যার ও দুর্যোগ এর মোকাবিলা করার প্রাথমিক দায়িত্ব দেশের সরকারের বিশেষত আমাদের দেশে যখন জনকল্যাণমুখী গণতান্ত্রিক রাষ্ট্র প্রতিষ্ঠিত। কিন্তু কী অর্থে ,কি সামর্থ্যে এই বিরাট সমস্যার সমাধান করা রাষ্ট্রশক্তির একার পক্ষে কখনোই সম্ভব নয়। সেই দিক দিয়ে এদেশের সমাজ সেবার গুরুত্ব অপরিসীম।"
#sentence4= "সুপ্রাচীন কাল থেকেই ভারতীয় উপমহাদেশ অর্থনৈতিক সমৃদ্ধি ও সাংস্কৃতিক ঐতিহ্যের জন্য সুপরিচিত। ঐতিহাসিক সিন্ধু সভ্যতা এই অঞ্চলেই গড়ে উঠেছিল। ইতিহাসের বিভিন্ন পর্বে এখানেই স্থাপিত হয়েছিল বিশালাকার একাধিক সাম্রাজ্য। নানা ইতিহাস-প্রসিদ্ধ বাণিজ্যপথ এই অঞ্চলের সঙ্গে বিশ্বের অন্যান্য সভ্যতার বাণিজ্যিক ও সাংস্কৃতিক সম্পর্ক রক্ষা করত। হিন্দু, বৌদ্ধ, জৈন ও শিখ—বিশ্বের এই চার ধর্মের উৎসভূমি ভারত। খ্রিস্টীয় প্রথম সহস্রাব্দে জরথুস্ত্রীয় ধর্ম (পারসি ধর্ম), ইহুদি ধর্ম, খ্রিস্টধর্ম ও ইসলাম ধর্ম এ দেশে প্রবেশ করে, ও ভারতীয় সংস্কৃতিতে বিশেষ প্রভাব বিস্তার করে। অষ্টাদশ শতাব্দীর প্রথমার্ধ থেকে ব্রিটিশ ইস্ট ইন্ডিয়া কোম্পানি ধীরে ধীরে ভারতীয় ভূখণ্ডের অধিকাংশ অঞ্চল নিজেদের শাসনাধীনে আনতে সক্ষম হয়। উনবিংশ শতাব্দীর মধ্যভাগে এই দেশ পুরোদস্তুর একটি ব্রিটিশ উপনিবেশে পরিণত হয়ে ওঠে। অতঃপর, এক সুদীর্ঘ স্বাধীনতা সংগ্রামের মধ্য দিয়ে ১৯৪৭ সালে ভারত একটি স্বতন্ত্র রাষ্ট্ররূপে আত্মপ্রকাশ করে। ১৯৫০ সালে সংবিধান প্রণয়নের মাধ্যমে ভারত একটি সার্বভৌম গণতান্ত্রিক প্রজাতন্ত্রে পরিণত হয়।"
#sentence5="এদেশের সমাজসেবার সমস্যাটির যেমন গরিষ্ঠ তেমনি এর ক্ষেত্রটি ও বহুদূর বিস্তৃত। এই বিশাল ক্ষেত্রে সমাজ সেবায় অংশগ্রহণ করবে কে বা কারা? অবশ্য সরকারের ভূমিকায় এতে অগ্রগণ্য ,কিন্তু দেশের অসংখ্য জনহিতকর সেবা প্রতিষ্ঠান গুলির ভূমিকা ও এতে নগণ্য নয়। সমাজসেবামূলক এই প্রতিষ্ঠান গুলির মধ্যে রামকৃষ্ণ মিশন, ভারত সেবাশ্রম, মাড়োয়ারি রিলিফা সোসাইটি, রেড ক্রস, সেন্ট জনস্ অ্যাম্বুলেন্স প্রভৃতি উল্লেখযোগ্য এবং তাদের অবদান তাদের অকুণ্ঠ ও নিরলস জনসেবা প্রশংসনীয়। তবে সমাজসেবার বিশাল ক্ষেত্র ও বিপুল গুরুত্বের তুলনায় এই সেবা উল্লেখযোগ্য হলেও পর্যাপ্ত নয়।সমাজসেবার দায়িত্ব পালন ও গুরু ভার বহনের জন্য তাই প্রয়োজন বৃহত্তর শক্তির।"
#sentence6="পাঠিকা ঠাকুরানি নিজ গুণে ক্ষমা করবেন। মাতালের গল্প ছাড়া আমার গতি নেই।"
#sentence7="তাই শুধু স্বাভাবিকভাবে নয়, প্রয়োজনের কারণে ও দেশের সুবিশাল ছাত্রসমাজের সুবিপুল কর্মশক্তির ওপর সমাজসেবার অংশগ্রহণের কথা এসে পড়ে। এখানে ও সমাজ সেবায় ছাত্র দের সম্পর্ক। ছাত্রসমাজ শুধু শক্তি মান নয়,সমাজসেবার কাছে তারা যোগ্য ও উপযুক্ত ও বটে। কারণ সমাজ সেবার জন্য যে সময় এবং শ্রম এর প্রয়োজন, আগ্রহ থাকলেও নানান সমস্যায় নিত্য বিব্রত কর্মজীবী সংসারী মানুষ তা দিতে পারে না। আপাতত দৃষ্টিতে অসম্ভব হলেও দেশের ছাত্রসমাজই সমাজসেবার মহান দায়িত্ব পালনে সময় ও শ্রম নিয়োগ করতে সক্ষম।অসীম প্রাণশক্তিতে উদ্বেল, অফুরন্ত উৎসাহে চঞ্চল, উচ্ছৃসিত আনন্দে ভরপুর তাজা তরুণ ছাত্র সমাজ তাই যেকোনো দুঃসাহসী কাজে নিঃস্বার্থ পরোপকারিতায় ঝাঁপিয়ে পড়ার জন্য সদাই উন্মুখ। লেখাপড়া ও খেলাধুলা করেও ছাত্রদের যে উৎসাহ উদ্বৃত থাকে তাকেই সমাজসেবার কাজে নিয়োজিত করতে হবে। ছাত্রজীবনে সমাজসেবার এই গুরুত্ব ও প্রয়োজনীয়তার কথা বিবেচনা করেই আমাদের মাধ্যমিক স্তরের শিক্ষাক্রমে সমাজসেবাকে পাঠ্য ও আচরণীয় বিষয় হিসেবে গ্রহণ করা হয়েছে"
# sentence8="তাই শুধু স্বাভাবিকভাবে নয়, প্রয়োজনের কারণে ও দেশের সুবিশাল ছাত্রসমাজের সুবিপুল কর্মশক্তির ওপর সমাজসেবার অংশগ্রহণের কথা এসে পড়ে। এখানে ও সমাজ সেবায় ছাত্র দের সম্পর্ক। ছাত্রসমাজ শুধু শক্তি মান নয়,সমাজসেবার কাছে তারা যোগ্য ও উপযুক্ত ও বটে। কারণ সমাজ সেবার জন্য যে সময় এবং শ্রম এর প্রয়োজন, আগ্রহ থাকলেও নানান সমস্যায় নিত্য বিব্রত কর্মজীবী সংসারী মানুষ তা দিতে পারে না। আপাতত দৃষ্টিতে অসম্ভব হলেও দেশের ছাত্রসমাজই সমাজসেবার মহান দায়িত্ব পালনে সময় ও শ্রম নিয়োগ করতে সক্ষম।অসীম প্রাণশক্তিতে উদ্বেল, অফুরন্ত উৎসাহে চঞ্চল, উচ্ছৃসিত আনন্দে ভরপুর তাজা তরুণ ছাত্র সমাজ তাই যেকোনো দুঃসাহসী কাজে নিঃস্বার্থ পরোপকারিতায় ঝাঁপিয়ে পড়ার জন্য সদাই উন্মুখ। লেখাপড়া ও খেলাধুলা করেও ছাত্রদের যে উৎসাহ উদ্বৃত থাকে তাকেই সমাজসেবার কাজে নিয়োজিত করতে হবে। ছাত্রজীবনে সমাজসেবার এই গুরুত্ব ও প্রয়োজনীয়তার কথা বিবেচনা করেই আমাদের মাধ্যমিক স্তরের শিক্ষাক্রমে সমাজসেবাকে পাঠ্য ও আচরণীয় বিষয় হিসেবে গ্রহণ করা হয়েছে"
sentence9="অক্ষর জ্ঞান যার নেই তাকেই নিরক্ষর বলা যেতে পারে। নিরক্ষর মানুষ জীবনের সবদিক থেকেই পিছিয়ে নিরক্ষরতা বহু ক্ষেত্রে মানুষের সার্বিক আত্মিক বিকাশের অন্তরায়। নিরক্ষরতার অভিশাপে অভিশপ্ত ব্যক্তি সাধারণতঃ নিজের অথবা সমাজের জন্য কিছু করে দেখাতে পারেনা। নিরক্ষরতার পরিণতি হলো নিরবতার অনুসরণ করা অর্থাৎ প্রত্যেক ক্ষেত্রে পিছিয়ে পড়া। নিরক্ষর মানুষ পৃথিবীকে নিজেকে পরিবেশকে লাঞ্ছনা কে ও শোষণ কে জানতে পারেনা। রবীন্দ্রনাথ ঠাকুর বলেছেন মানুষের অন্ধত্বের মত নিরক্ষরতা এই দুর্ভাগ্য দেশের হতভাগ্য জনগণের সর্বাপেক্ষা নিষ্ঠুরতম অভিশাপ। নিরক্ষরতার স্বরূপকে বোঝার জন্য আমরা আমাদের দেশ ভারত কে বুঝলেই চলবে- যেখানে মোট জনসংখ্যার এক-তৃতীয়াংশ নিরক্ষর। এই নিরক্ষতাকে দূর করা একান্ত প্রয়োজনীয়।"
sentences=sentence9.replace("-"," ")

sentences=sentences.split("।")

for sentence in sentences:
    words.extend(nltk.word_tokenize(sentence))

In [38]:
for i in words:
    if len(i)<2:
        words.remove(i)
for word in words:
    search_word = word

    with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        found_root_word = False
        #check if the word exists in dataset
        for row in reader:
            if len(row) > 0 and row[0] == search_word:
                print(f"{search_word} -> {row[0]}")
                found_root_word = True
            elif len(row) > 0 and search_word in row:
                print(f"{search_word} -> {row[0]}")
                found_root_word = True
                
        if not found_root_word:

            if words.index(word)==0:
                left_context=''
                right_context=words[words.index(word)+1]
            elif words.index(word)==len(words)-1:
                left_context=words[words.index(word)-1]
                right_context=''
            else:
                left_context=words[words.index(word)-1]
                right_context=words[words.index(word)+1]
            sub_dictnary=slice_dictnary(word)
            if not sub_dictnary.empty:
                prediction_list=[]
                leematizer(word)    
            else :
                print (word,"not in database")        

অক্ষর -> অক্ষর
input  অক্ষর
অক্ষর ---> অক্ষর
Added অক্ষর as a related word for অক্ষর
জ্ঞান -> জ্ঞান
input  জ্ঞান
জ্ঞান ---> জ্ঞান
Added জ্ঞান as a related word for জ্ঞান
input  যার
যার ---> যে
Added যার as a related word for যে
নেই -> নেই
input  নেই
নেই ---> নেই
Added নেই as a related word for নেই
input  তাকেই
তাকেই ---> তাকা
Added তাকেই as a related word for তাকানো
নিরক্ষর -> নিরক্ষর
input  নিরক্ষর
নিরক্ষর ---> নিরক্ষর
Added নিরক্ষর as a related word for নিরক্ষর
বলা -> বলা
input  বলা
বলা ---> বলা
Added বলা as a related word for বলা
যেতে -> যেতে
input  যেতে
যেতে ---> যেতে
Added যেতে as a related word for যে
input  পারে
পারে ---> পরা
Added পারে as a related word for পরা
নিরক্ষর -> নিরক্ষর
input  নিরক্ষর
নিরক্ষর ---> নিরক্ষর
Added নিরক্ষর as a related word for নিরক্ষর
মানুষ -> মানুষ
input  মানুষ
মানুষ ---> মানুষ
Added মানুষ as a related word for মানুষ
input  জীবনের
জীবনের ---> জীবন
Added জীবনের as a related word for জীবন
সবদিক -> সবদিক
input  সবদিক
সবদিক ---> সবদিক
Added সবদিক as a relat

In [39]:
prediction_list=[]
container=[]
sub_dictnary=slice_dictnary('উঠলো')
leematizer('উঠলো')
translation_window=translator.translate('সে আমাকে বলেছিল').lower()
print(translation_window)
for translation_testword in ['আমি', 'আম']:
    translation_testword=translator.translate('সে '+translation_testword+' বলেছিল').lower()
    print(translation_testword)
    print(LCsequence(translation_window,translation_testword)/len(translation_testword))
#     print(LCsequence(translation_window,translation_testword,len(translation_window),len(translation_testword)))
#    print(LCsequence(translation_window,translation_testword,len(translation_window),len(translation_testword))/len(translation_testword))
#     print(LCsequence(translation_window,translation_testword,len(translation_window),len(translation_testword))/abs(len(translation_testword)-len(translation_window)))
    print(abs(len(translation_testword)-len(translation_window))/LCsequence(translation_window,translation_testword))

input  উঠলো
উঠলো ---> ওঠা
Added উঠলো as a related word for ওঠা
he told me
he told me
1.0
0.0
he said amen.
0.5384615384615384
0.42857142857142855


In [40]:
prediction_list=[]
container=[]
leematizer('গায়ের')

translation_window=translator.translate('তার গায়ের রঙ ').lower()
print(translation_window)
for translation_testword in ['গাওয়া', 'গা']:
    translation_testword=translator.translate('তারে '+translation_testword+' রঙ').lower()
    print(translation_testword)
# print(LCsequence(translation_window,translation_testword,len(translation_window),len(translation_testword)))
    print(LCsequence(translation_window,translation_testword)/len(translation_testword))
#     print(LCsequence(translation_window,translation_testword,len(translation_window),len(translation_testword))/abs(len(translation_testword)-len(translation_window)))
    print(abs(len(translation_testword)-len(translation_window))/LCsequence(translation_window,translation_testword))

input  গায়ের
গায়ের not in database
the color of his skin.
the color he sings.
0.8421052631578947
0.1875
color of hers
0.8461538461538461
0.8181818181818182


In [41]:
translation_testword=translator.translate('তারে গাওয়া রঙ').lower()
translation_window='the color of his skin'
if translation_testword:
    print(translation_testword)
print(abs(len(translation_testword)-len(translation_window))/LCsequence(translation_window,translation_testword))

the color he sings.
0.13333333333333333
